In [4]:
import os
import quinn
from pyspark.ml.feature import VectorAssembler, Normalizer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml import Pipeline
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

# Initialize Spark context and session
sc = SparkContext('local')
spark = SparkSession(sc)

# Load datasets
train_df = spark.read.format('csv').options(header='true', inferSchema='true', sep=';').load('/home/ubuntu/cs643rudolphpaulin/TrainingDataset.csv')
test_df = spark.read.format('csv').options(header='true', inferSchema='true', sep=';').load('/home/ubuntu/cs643rudolphpaulin/ValidationDataset.csv')

print("Data loaded into Spark.")
print(train_df.toPandas().head())

# Remove quotations and rename "quality" to "label"
def remove_quotations(s):
    return s.replace('"', '')

train_df = quinn.with_columns_renamed(remove_quotations)(train_df)
train_df = train_df.withColumnRenamed('quality', 'label')

test_df = quinn.with_columns_renamed(remove_quotations)(test_df)
test_df = test_df.withColumnRenamed('quality', 'label')

print("Data has been formatted.")
print(train_df.toPandas().head())

# Feature engineering
assembler = VectorAssembler(
    inputCols=["fixed acidity",
               "volatile acidity",
               "citric acid",
               "residual sugar",
               "chlorides",
               "free sulfur dioxide",
               "total sulfur dioxide",
               "density",
               "pH",
               "sulphates",
               "alcohol"],
    outputCol="inputFeatures"
)

scaler = Normalizer(inputCol="inputFeatures", outputCol="features")

# Logistic Regression model
lr = LogisticRegression()

# Pipeline for data processing and model training
pipeline1 = Pipeline(stages=[assembler, scaler, lr])

# Parameter grid and evaluator
paramgrid = ParamGridBuilder().build()
evaluator = MulticlassClassificationEvaluator(metricName="f1")

# Cross-validation
crossval = CrossValidator(estimator=pipeline1,  
                          estimatorParamMaps=paramgrid,
                          evaluator=evaluator, 
                          numFolds=3)

cvModel1 = crossval.fit(train_df)

# Evaluate model
f1_score = evaluator.evaluate(cvModel1.transform(test_df))
print("F1 Score for Logistic Regression Model: ", f1_score)

# Save the trained model
model_path = "/home/ubuntu/LogisticRegressionModel"

def ensure_directory_exists(directory_path):
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)
        print(f"Directory created: {directory_path}")
    else:
        print(f"Directory already exists: {directory_path}")

ensure_directory_exists(model_path)
cvModel1.bestModel.write().overwrite().save(model_path)
print(f"Model saved to: {model_path}")


/home/ec2-user/.local/lib/python3.7/site-packages/pyspark/bin/spark-class: line 71: /usr/bin/java/bin/java: Not a directory
/home/ec2-user/.local/lib/python3.7/site-packages/pyspark/bin/spark-class: line 97: CMD: bad array subscript


RuntimeError: Java gateway process exited before sending its port number